In [ ]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

def residuals(t, x, y):
    """Return an array of all the residuals"""
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions

## Regression Model

In [ ]:
#This code will generate an example data set based on a given slope and intercept. 
#The amount of data points is set by the sample sizxe.

def draw_and_compare(true_slope, true_int, sample_size):
    x = np.random.normal(50, 5, sample_size)
    xlims = np.array([np.min(x), np.max(x)])
    errors = np.random.normal(0, 6, sample_size)
    y = (true_slope * x + true_int) + errors
    sample = Table().with_columns('x', x, 'y', y)

    sample.scatter('x', 'y')
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title('True Line, and Points Created')

    sample.scatter('x', 'y')
    plots.title('What We Get to See')

    sample.scatter('x', 'y', fit_line=True)
    plots.title('Regression Line: Estimate of True Line')

    sample.scatter('x', 'y', fit_line=True)
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title("Regression Line and True Line")

In [ ]:
#Identify the slope, intercept, and sample size.
#How would you write the equation of the true line?

draw_and_compare(2, -5, 10)



### Look at the last graph. How well did the Regression Line predict the true line? ###


In [ ]:
#Identify the slope, intercept, and sample size.
#How would you write the equation of the true line?

draw_and_compare(2, -5, 25)


### Look at the last graph. How well did the Regression Line predict the true line? ###


In [ ]:
#Identify the slope, intercept, and sample size.


draw_and_compare(2, -5, 100)


### Look at the last graph. How well did the Regression Line predict the true line? ###


In [ ]:
#Identify the slope, intercept, and sample size.

draw_and_compare(2, -5, 1000)





### Look at the last graph. How well did the Regression Line predict the true line? ###
Compare this prediction to the previous predictions. What made them different?

## Prediction

In [ ]:
#Import the births data set. 

births = Table.read_table('baby.csv')
births.show(3)

In [ ]:
# Preterm and postterm pregnancy cutoffs, according to the CDC in days (converted from weeks).
37 * 7, 42 * 7

In [ ]:
#Create a scatter comparing Gestational Days to Birth Weight



In [ ]:
#Create a new table births with Gesational Days between 225 and 325.



In [ ]:
#Create a scatter comparing Gestational Days to Birth Weight of the new births table. 



## Suppose we assume the regression model

If the data come from the regression model,
* The “true value” of the response y at a given value of x is the height of the true line at x
* We can’t see the true line, so we have to estimate this height
* The regression line is most likely close to true line
* Given a new value of x, predict y by finding the point on the regression line at that x


In [ ]:
# Find the correlation between Gestational Days and Birth Weight



In [ ]:
# Create a scatter comparing Gestational Days and Birth Weight that also displays the Fit Line. 



## Prediction at a Given Value of x

What does the function below do?


In [ ]:
def prediction_at(t, x, y, x_value):
    '''
    t - table
    x - label of x column
    y - label of y column
    x_value - the x value for which we want to predict y
    '''
    return slope(t, x, y) * x_value + intercept(t, x, y)

In [ ]:
# Use the function to find the Birth Weight prediction at 300 Gestational Days. 

prediction_at_300 = 
prediction_at_300

### What does the predicted value above represent? What was the input? ###

In [ ]:
# Plot the predicted value on the scatter.

x = 300
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.plot([x, x], [40, prediction_at_300], color='red', lw=2);

## Bootstrapping the Sample

Refresh: What does bootstrapping do?

What will the scatters below compare?

In [ ]:
# This plots several bootstrap samples based on the original sample. 

plots.figure(figsize=(10, 11))
plots.subplot(3, 2, 1)
plots.scatter(births[1], births[0], s=10, color='darkblue')
plots.xlim([225, 325])
plots.title('Original sample')

for i in np.arange(1, 6, 1):
    plots.subplot(3,2,i+1)
    resampled = births.sample()
    plots.scatter(resampled.column('Gestational Days'), resampled.column('Birth Weight'), s=10, color='tab:green')
    plots.xlim([225, 325])
    plots.title('Bootstrap sample '+str(i))
plots.tight_layout()

In [ ]:
#Each bootstrap sample will generate a different predicted value. 

for i in np.arange(4):
    resample = births.sample()
    predicted_y = prediction_at(resample, 'Gestational Days', 'Birth Weight', 300)
    print('Predicted y from bootstramp sample was', predicted_y)
    resample.scatter('Gestational Days', 'Birth Weight', fit_line=True)
    plots.scatter(300, predicted_y, color='gold', s=50, zorder=3);
    plots.plot([x, x], [40, predicted_y], color='red', lw=2);
    plots.plot([200, x], [predicted_y, predicted_y], color='red', lw=2);

In [ ]:
#This finds the Regression line for each bootstrap sample and plots on the same coordinate plane. 
#It also marks the prediction at 300 days on each line. 

lines = Table(['slope','intercept', 'at 210', 'at 300', 'at 320'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 210 + b, a * 300 + b, a * 320 + b])

for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plots.plot([210, 320], [line.item('at 210'), line.item('at 320')], lw=1)
    plots.scatter(300, line.item('at 300'), s=30, zorder=3)
    


### Where do the lines seem to intersect or are the closest to each other? ###

In [ ]:
#Find and display the mean of the gestational days and the mean of the birth weight.

np.mean(births.column('Gestational Days')), np.mean(births.column('Birth Weight'))



### Compare the result to where the lines seem to intersect. ###

In [ ]:
#Plotting lines again but zooming in on the area we are interested in. 
#What are the differences in this code compared to the previous?

lines = Table(['slope','intercept', 'at 291', 'at 300', 'at 309'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 291 + b, a * 300 + b, a * 309 + b])
lines


### Interpret the slope and y-intercept in context ###
Choose one of the rows and write out the regression line model. Then interpret the slope and intercept in context of the data. 


In [ ]:
#A zoomed in view focusing on the predictions at 300 days. 

for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plots.plot([291, 309], [line.item('at 291'), line.item('at 309')], lw=1)
    plots.scatter(300, line.item('at 300'), s=30, zorder=3)

## Prediction Interval

Bootstrap the scatter plot
 * Get a prediction for y using the regression line that goes through the resampled plot
 * Repeat the two steps above many times
 * Draw the empirical histogram of all the predictions.
 * Get the “middle 95%” interval. This is our range of predictions of y.
 * It is an approximate 95% confidence interval for the height of the true line at x.

In [ ]:
# This code will generate many bootstrap samples, find the prediction in each of those samples,
# find the endpoints of the 95% interval, and display the results as a histogram. 

def bootstrap_prediction(t, x, y, new_x, repetitions=2500):
    """ 
    Makes a 95% confidence interval for the height of the true line at new_x, 
    using linear regression on the data in t (column names x and y).
    Shows a histogram of the bootstrap samples and shows the interval
    in gold.
    """

    # Bootstrap the scatter, predict, collect
    predictions = make_array()
    for i in np.arange(repetitions):
        resample = t.sample()
        predicted_y = prediction_at(resample, x, y, new_x)
        predictions = np.append(predictions, predicted_y)

    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)
    round_left = round(left, 3)
    round_right = round(right, 3)

    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20)
    plots.xlabel('predictions at x='+str(new_x))
    plots.plot([left, right], [0, 0], color='yellow', lw=8);
    print('Approximate 95%-confidence interval for height of true line at x =', new_x)
    print(round_left, 'to', round_right, '( width =', round(right - left, 3), ')') 

In [ ]:
#Use the function to find the 95% confidence interval for Birth Weight when the Gestiational Dats are  300. 



### What is the spread of the confidence interval? What does that mean? ###


## Predictions at Different Values of x

Since y is correlated with x, the predicted values of y depend on the value of x.


The width of the prediction interval also depends on x.</br>
Typically, intervals are wider for values of x that are further away from the mean of x.

In [ ]:
#Plot the predication on the scatter when x = 300

x = 300
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.plot([x, x], [40, prediction_at_300], color='red', lw=2);



In [ ]:
#When x = 230

bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 230)



### Record the spread of the confidence interval. ###

In [ ]:
#When x = 280

bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 280)



Record the spread of the confidence interval.

In [ ]:
#Recall the mean of both gestational days and birth weight. 

np.mean(births.column('Gestational Days')), np.mean(births.column('Birth Weight'))



### Which of the above graphs is closest to these values? What is the spread of the confidence interval on that graph? ###

In [ ]:
#Graph lines at different x values again without marking a particular prediction.

lines = Table(['slope','intercept', 'at 210', 'at 300', 'at 320'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 210 + b, a * 300 + b, a * 320 + b])

for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plots.plot([210, 320], [line.item('at 210'), line.item('at 320')], lw=1)

### What observations can you make? ###

## Inference for the True Slope

To find the Confidence Interval for the Slope of the True Line:
 * Bootstrap the scatter plot.
 * Find the slope of the regression line through the bootstrapped plot.
 * Repeat.
 * Draw the empirical histogram of all the generated slopes.
 * Get the “middle 95%” interval.
 * That’s an approximate 95% confidence interval for the slope of the true line. 

In [ ]:
#Show the scatter of births with the True Line. 

births.scatter('Gestational Days', 'Birth Weight', fit_line=True)

In [ ]:
# Determine the true slope and intercept of the data. 

slope(births, 'Gestational Days', 'Birth Weight'), intercept(births, 'Gestational Days', 'Birth Weight')

In [ ]:
#Create bootstrap samples, find the slopes of each, generate the end points of the confidence interval,
#and display the results. 

def bootstrap_slope(t, x, y, repetitions=2500):
    """ 
    Makes a 95% confidence interval for the slope of the true line, 
    using linear regression on the data in t (column names x and y).
    Shows a histogram of the bootstrap samples and shows the interval
    in gold.
    """
    
    # Bootstrap the scatter, find the slope, collect
    slopes = make_array()
    for i in np.arange(repetitions):
        bootstrap_sample = t.sample()
        bootstrap_slope = slope(bootstrap_sample, x, y)
        slopes = np.append(slopes, bootstrap_slope)
    
    # Find the endpoints of the 95% confidence interval for the true slope
    left = percentile(2.5, slopes)
    right = percentile(97.5, slopes)
    round_left = round(left, 3)
    round_right = round(right, 3)
    
    # Slope of the regression line from the original sample
    observed_slope = slope(t, x, y)
    
    # Display results (no need to follow this code)
    Table().with_column('Bootstrap Slopes', slopes).hist(bins=20)
    plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=8);
    print('Slope of regression line:', round(observed_slope, 3))
    print('Approximate 95%-confidence interval for the slope of the true line:')
    print(round_left, 'to', round_right)

In [ ]:
#Use the function to graph the Confidence Interval for the slope for Gestational Days anbased on the bootstraps. 

bootstrap_slope(births, 'Gestational Days', 'Birth Weight')

## Rain on the Regression Parade

What if the slope of the true line is actually zero?

What would that mean for the correlation?

In [ ]:
draw_and_compare(0, 10, 25)

### What is the problem here? ### 

### Maternal Age and Birth Weight

In [ ]:
births.scatter('Maternal Age', 'Birth Weight')

In [ ]:
#Find the slope of the true line for our data. 

slope(births, 'Maternal Age', 'Birth Weight')

In [ ]:
births.scatter('Maternal Age', 'Birth Weight', fit_line=True)

### Test whether there really is a slope. 

**Null:** Slope of true line is equal to 0.

**Alternative:** Slope of true line is not equal to 0.

Method:
 * Construct a bootstrap confidence interval for the true slope.
 * If the interval doesn’t contain 0, reject the null hypothesis.
 * If the interval does contain 0, there isn’t enough evidence to reject the null hypothesis.

In [ ]:
bootstrap_slope(births, 'Maternal Age', 'Birth Weight')

### Would we reject the null or is the data consistent with the null? Explain your choice. ###